## データ取得

In [5]:
# bitflyerからティッカーを取得
import requests
response = requests.get("https://api.bitflyer.jp/v1/ticker/")
# json形式に変換
data = response.json()
print(data)
print(data['best_bid'])

{'product_code': 'BTC_JPY', 'state': 'RUNNING', 'timestamp': '2022-01-19T03:10:41.753', 'tick_id': 41367778, 'best_bid': 4852095.0, 'best_ask': 4854107.0, 'best_bid_size': 0.1, 'best_ask_size': 0.05, 'total_bid_depth': 566.25160929, 'total_ask_depth': 516.23615341, 'market_bid_size': 0.0, 'market_ask_size': 0.0, 'ltp': 4853000.0, 'volume': 8355.53627651, 'volume_by_product': 2262.95366543}


In [10]:
# 期間を指定してcryptowatchからデータを取得
response = requests.get('https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc?periods=86400&after=1514764800')
response

<Response [200]>

In [36]:
min_ = 10**9
for num, val in enumerate(response['result']['86400']):
    # print(num, val, val[3])
    min_ = min(min_, val[3])
print(min_)

354350


## 指値注文

In [ ]:
# api経由で指し値注文を入れる
# ccxtを使えば簡単だが、勉強用に書いたコード
import hashlib
import hmac
import requests
import datetime
import json

api_key = ''
api_secret = ''

base_url = 'https://api.bitflyer.jp'
path_url = '/v1/me/sendchildorder'
method = 'POST'

timestamp = str(datetime.datetime.today())

param = {
	"product_code" : "FX_BTC_JPY",
	"child_order_type" : "LIMIT",
	"side" : "BUY",
	"price" : 指値,
	"size" : 0.01,
}
body = json.dumps(params)

message = timestamp + method + path_url + body
signature = hmac.new(bytearray(api_secret.encode('utf-8')), message.encode('utf-8'), digestmod = hashlib.sha256).hexdigest()

headers = {
	'ACCESS-KEY' : api_key,
	'ACCESS-TIMESTAMP' : timestamp,
	'ACCESS-SIGN' : signature,
	'Content-Type' : 'application/json'
}

response = requests.post(base_url + path_url, data=body, headers = headers )
print(response.status_code)
print(response.json())

## ティッカー情報取得

In [1]:
# ccxtでティッカー情報を取得
import ccxt
from pprint import pprint

bitflyer = ccxt.bitflyer()
ticker = bitflyer.fetch_ticker('BTC/JPY', params = {'product_code' : 'FX_BTC_JPY'} )
pprint(ticker)

{'ask': 4425592.0,
 'askVolume': None,
 'average': None,
 'baseVolume': 8066.46025461,
 'bid': 4424499.0,
 'bidVolume': None,
 'change': None,
 'close': 4425692.0,
 'datetime': '2022-01-28T21:52:22.897Z',
 'high': None,
 'info': {'best_ask': '4425592.0',
          'best_ask_size': '0.02',
          'best_bid': '4424499.0',
          'best_bid_size': '0.04',
          'ltp': '4425692.0',
          'market_ask_size': '0.0',
          'market_bid_size': '0.0',
          'product_code': 'FX_BTC_JPY',
          'state': 'RUNNING',
          'tick_id': '13551749',
          'timestamp': '2022-01-28T21:52:22.897',
          'total_ask_depth': '170.02476421',
          'total_bid_depth': '397.81560699',
          'volume': '8070.80825461',
          'volume_by_product': '8066.46025461'},
 'last': 4425692.0,
 'low': None,
 'open': None,
 'percentage': None,
 'previousClose': None,
 'quoteVolume': None,
 'symbol': 'BTC/JPY',
 'timestamp': 1643406742897,
 'vwap': None}


## FXの証拠金残高を確認
ccxtには、Bitflyerで BTC-FX の証拠金残高を取得する共通の関数がない  
ccxt経由で直接apiを操作して情報を取得するには、以下の用にするして関数を作る
取引所のAPIを使うためのコード  
X）プライベートAPIかパブリックAPIか？  
Y）メソッドはGET方式か？POST方式か？  
Z）APIのリクエストURL（パス）は？  
->> 関数名 ： .X_Y_Z()  
https://ryota-trade.com/?p=629  

In [ ]:
# 取引所のAPIを使うためのコード
# X）プライベートAPIかパブリックAPIか？
# Y）メソッドはGET方式か？POST方式か？
# Z）APIのリクエストURL（パス）は？
# ->> 関数名 ： .X_Y_Z()
# https://ryota-trade.com/?p=629
import ccxt
from pprint import pprint

bitflyer = ccxt.bitflyer()
bitflyer.apiKey = ''
bitflyer.secret = ''

collateral = bitflyer.private_get_getcollateral()
pprint(collateral)

In [43]:
# 取引所が対応している共通のcopy_contextの関数
import ccxt
bitflyer = ccxt.bitflyer()
method_list = bitflyer.has

for key, val in method_list.items():
    if val == True:
        print(key, val)

publicAPI True
privateAPI True
cancelOrder True
createLimitOrder True
createMarketOrder True
createOrder True
fetchBalance True
fetchL2OrderBook True
fetchMarkets True
fetchMyTrades True
fetchOrderBook True
fetchOrders True
fetchPositions True
fetchTicker True
fetchTrades True
loadMarkets True
withdraw True


In [ ]:
# 注文情報の取得
import ccxt
from pprint import pprint

bitflyer = ccxt.bitflyer()
bitflyer.apiKey = ''
bitflyer.secret = ''

# open_order:まだ約定していない注文のこと
order = bitflyer.fetch_open_orders(
    symbol = 'BTC/JPY', 
    params = {'product_code', 'FX_BTC_JPY'}
)
pprint(orders)

In [ ]:
# 注文情報のID取得
import ccxt
from pprint import pprint

bitflyer = ccxt.bitflyer()
bitflyer.apiKey = ''
bitflyer.secret = ''

orders = bitflyer.fetch_open_orders(
    symbol = 'BTC/JPY',
    params = {'product_code' : 'FX_BTC_JPY'}
 )

 for o in orders:
     pprint(o['id'])

In [ ]:
# 注文情報の状況を取得
import ccxt
from pprint import pprint

bitflyer = ccxt.bitflyer()
bitflyer.apikey = ''
bitflyer.secret = ''

orders = bitflyer.fetch_orders(
    symbol = 'BTC/JPY',
    params = {'product_code' : 'FX_BTC_JPY', 'count' : 10}
)

for o in orders:
    print(o['id'] + ' : 注文状況 ' + o['status'])

In [ ]:
# 注文のキャンセル
bitflyer.cancel_order(
    symbol = 'BTC/JPY',
    id = ''
    params = {'product_code' : 'FX_BTC_JPY'}
)

In [ ]:
# Cryptowatchからbitflyerのデータを取得
# ローソク足を取得
import requests
from datetime import datetime
import time

# 1分足
response = requests.get('https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods=60')
response = response.json()

In [4]:
# 一つ前のローソク足を取得（終値が確定したものの最新）
# [-1]だと、終値がまだ確定していない
data = response['result']['60'][-2]

close_time = datetime.fromtimestamp(data[0]).strftime('%Y/%m/%d %H:%M')
open_price = data[1]
close_price = data[4]

print("時間: " + close_time
	+ " 始値: " + str(open_price)
	+ " 終値: " + str(close_price) )

時間： 2022/01/20 13:05 始値： 4937280 終値： 4936938


In [ ]:
# 10秒でデータを取って繰り返す。
import requests
from datetime import datetime
import time

while True:
    response = response.get('https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods=60')
    
    data = response['result']['60'][-2]

    close_time = datetime.fromtimestamp(data[0]).strftime('%Y/%m/%d %H:%M')
    open_price = data[1]
    close_price = data[4]

    print("時間: " + close_time
        + " 始値: " + str(open_price)
        + " 終値: " + str(close_price))

    time.sleep(10)

In [8]:
# 上にループを抜けるコードがないと、以下のコードはグレーアウトする
y = 1 + 2
print(y)

3


In [5]:
# 更新された時だけprintする
import requests
from datetime import datetime
import time


response = requests.get("https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods=60")
response = response.json()

# 前回の時間を last_time に入れる
last_data = response["result"]["60"][-2]
last_time = datetime.fromtimestamp(last_data[0]).strftime('%Y/%m/%d %H:%M')
time.sleep(10)

while True:
    response = requests.get("https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods=60")
    response = response.json()

    data = response['result']['60'][-2]

    close_time = datetime.fromtimestamp(data[0]).strftime('%Y/%m/%d %H:%M')
    open_price = data[1]
    close_price = data[4]
    if close_time != last_time:
        print("時間: " + close_time
			+ " 始値: " + str(open_price)
			+ " 終値: " + str(close_price))

        last_time = close_time

    time.sleep(10)

